<a href="https://colab.research.google.com/github/yalcindemir/Claude-to-Codellama-Distillation/blob/master/notebooks/Claude_Code_Model_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Claude-to-CodeLlama Knowledge Distillation

**Transform Claude Opus 4's Superior Code Generation into an Accessible 7B Model**

This notebook provides a complete end-to-end implementation of knowledge distillation from Claude Opus 4 to Code Llama 7B.

## 📋 Features
- 🧠 **Teacher-Student Learning**: Claude Opus 4 → Code Llama 7B
- 💰 **Cost Effective**: ~$50-100 for Colab Pro training
- ⚡ **Memory Efficient**: QLoRA optimization for 6GB GPU
- 📊 **Comprehensive Evaluation**: HumanEval and MBPP benchmarks
- 🔧 **Production Ready**: Save and deploy your trained model

## 🎯 Expected Results
- **HumanEval**: 70-75% pass@1 (vs 33.5% baseline)
- **MBPP**: 65-70% pass@1 (vs 41.4% baseline)
- **Training Time**: 4-6 hours on Colab Pro
- **Total Cost**: ~$60-80 including API calls

## 🛠️ Setup Environment

First, let's set up the environment and install dependencies.

In [23]:
# Check GPU availability
!nvidia-smi

# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

# Create project directory
import os
PROJECT_DIR = '/content/drive/MyDrive/claude_distillation'
os.makedirs(PROJECT_DIR, exist_ok=True)
os.chdir(PROJECT_DIR)

print(f"✅ Working directory: {os.getcwd()}")

Sat Jun 14 14:12:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
# Clone repository
if not os.path.exists('Claude-to-Codellama-Distillation'):
    !git clone https://github.com/yalcindemir/Claude-to-Codellama-Distillation.git
    print("✅ Repository cloned")
else:
    print("✅ Repository already exists")

# Removed os.chdir as the directory was not created due to the git clone failure.
# You will need to address the git clone error before proceeding.
# print(f"📂 Project directory: {os.getcwd()}")

✅ Repository already exists


In [25]:
# Install requirements
!pip install -r requirements.txt

# Additional Colab-specific packages
!pip install google-colab

print("✅ Requirements installed")

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
✅ Requirements installed


## 🔑 Configuration

Set up your API keys and configuration.

In [26]:
import os
from getpass import getpass

# Set API keys
print("🔑 Setting up API keys...")

# Claude API key (required)
if not os.getenv('ANTHROPIC_API_KEY'):
    anthropic_key = getpass('Enter your Anthropic API key: ')
    os.environ['ANTHROPIC_API_KEY'] = anthropic_key
    print("✅ Claude API key set")
else:
    print("✅ Claude API key already set")

# Weights & Biases (optional)
if not os.getenv('WANDB_API_KEY'):
    wandb_key = getpass('Enter your W&B API key (optional, press Enter to skip): ')
    if wandb_key:
        os.environ['WANDB_API_KEY'] = wandb_key
        print("✅ W&B API key set")
    else:
        print("⏭️ W&B skipped")
else:
    print("✅ W&B API key already set")

🔑 Setting up API keys...
✅ Claude API key already set
Enter your W&B API key (optional, press Enter to skip): ··········
✅ W&B API key set


In [27]:
# Colab-specific configuration
import torch
import sys

# Add src to path
sys.path.append('./src')

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎮 Device: {device}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name} ({gpu_memory:.1f}GB)")
else:
    print("⚠️ No GPU available - training will be very slow!")

# Colab-optimized configuration
COLAB_CONFIG = {
    'target_size': 1000,  # Small dataset for demo
    'num_epochs': 1,      # Quick training
    'batch_size': 2,      # Small batch for memory
    'max_length': 1024,   # Shorter sequences
    'use_4bit': True,     # QLoRA quantization
    'lora_r': 8,          # Smaller LoRA rank
}

print("✅ Colab configuration set")

🎮 Device: cuda
GPU: Tesla T4 (15.8GB)
✅ Colab configuration set


In [18]:
import os

# Doğru dizine geçin
os.chdir('/content/drive/MyDrive/claude_distillation/Claude-to-Codellama-Distillation')
print(f"✅ Yeni dizin: {os.getcwd()}")

# Şimdi src klasörünü kontrol edin
if os.path.exists('./src'):
    print("✅ ./src klasörü bulundu")
else:
    print("❌ src klasörü hala yok")
    # Dosya listesini görelim
    print("Mevcut dosyalar:")
    print(os.listdir('.'))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/claude_distillation/claude_to_codellama_distillation'

## 📊 Phase 1: Dataset Generation

Generate high-quality code examples using Claude Opus 4.

In [ ]:
import asyncio
from dataset_generator import DatasetGenerator, DatasetConfig
from claude_client import ClaudeConfig

# Configure dataset generation
claude_config = ClaudeConfig(
    api_key=os.getenv('ANTHROPIC_API_KEY'),
    model='claude-3-opus-20240229',
    max_tokens=1024,
    temperature=0.1,
    rate_limit_rpm=30  # Conservative for Colab
)

dataset_config = DatasetConfig(
    target_size=COLAB_CONFIG['target_size'],
    languages=['python', 'javascript'],  # Start with 2 languages
    output_dir='./data/generated'
)

print("🏗️ Starting dataset generation...")
print(f"Target size: {dataset_config.target_size} examples")
print(f"Languages: {dataset_config.languages}")

In [ ]:
# Run dataset generation
async def generate_dataset():
    generator = DatasetGenerator(dataset_config, claude_config)

    print('📊 Generating dataset...')
    dataset = await generator.generate_dataset(max_concurrent=2)

    if len(dataset) > 0:
        print(f'✅ Generated {len(dataset)} examples')

        # Split and save dataset
        dataset_dict = generator.split_dataset(dataset)
        generator.save_dataset(dataset_dict, format='jsonl')

        # Generate quality report
        report = generator.generate_quality_report()
        print(f'💰 Estimated cost: ${report["generation_summary"]["total_cost"]:.2f}')
        print(f'📈 Success rate: {report["quality_metrics"]["acceptance_rate"]:.2%}')

        return dataset_dict
    else:
        print('❌ No examples generated!')
        return None

# Run the generation
dataset_dict = await generate_dataset()

## 🎯 Phase 2: Model Training

Train Code Llama using knowledge distillation.

In [ ]:
from distillation_trainer import KnowledgeDistillationSystem, DistillationConfig
import wandb

# Initialize Weights & Biases (optional)
if os.getenv('WANDB_API_KEY'):
    wandb.init(
        project="claude-to-codellama-colab",
        config=COLAB_CONFIG,
        tags=["colab", "demo"]
    )
    print("✅ W&B initialized")

# Configure training
config = DistillationConfig(
    student_model_name='codellama/CodeLlama-7b-hf',
    dataset_path='./data/generated',
    output_dir='./models/distilled_codellama',
    max_length=COLAB_CONFIG['max_length'],
    num_epochs=COLAB_CONFIG['num_epochs'],
    batch_size=COLAB_CONFIG['batch_size'],
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    use_4bit=COLAB_CONFIG['use_4bit'],
    lora_r=COLAB_CONFIG['lora_r'],
    lora_alpha=16,
    use_gradient_checkpointing=True,
    eval_steps=50,
    save_steps=100,
    logging_steps=5
)

print("🎯 Training configuration ready")
print(f"Model: {config.student_model_name}")
print(f"Epochs: {config.num_epochs}")
print(f"Batch size: {config.batch_size}")
print(f"LoRA rank: {config.lora_r}")

In [ ]:
# Initialize training system
print("🚀 Initializing training system...")
system = KnowledgeDistillationSystem(config)

try:
    print("📚 Loading model and datasets...")
    results = system.run_full_training()

    print("🎉 Training completed successfully!")
    print(f"Final training loss: {results['train_result'].training_loss:.4f}")
    print(f"Final eval loss: {results['eval_results']['eval_loss']:.4f}")

    # Save training metrics
    training_metrics = {
        'train_loss': results['train_result'].training_loss,
        'eval_loss': results['eval_results']['eval_loss'],
        'epochs': config.num_epochs,
        'model_size': '7B',
        'lora_rank': config.lora_r
    }

    if wandb.run:
        wandb.log(training_metrics)

except Exception as e:
    print(f"❌ Training failed: {e}")
    print("This might be due to insufficient data or memory constraints.")
    print("Try reducing batch_size or dataset size.")

## 📈 Phase 3: Model Evaluation

Evaluate the trained model on standard benchmarks.

In [ ]:
from evaluation_system import ModelComparator, EvaluationConfig

# Configure evaluation
eval_config = EvaluationConfig(
    student_model_path='./models/distilled_codellama',
    baseline_models=['codellama/CodeLlama-7b-hf'],  # Compare with base model
    test_datasets=['humaneval'],  # Start with one benchmark
    output_dir='./evaluation_results',
    max_new_tokens=256,  # Shorter for speed
    temperature=0.1
)

print("📊 Evaluation configuration ready")
print(f"Datasets: {eval_config.test_datasets}")
print(f"Baseline models: {eval_config.baseline_models}")

In [ ]:
# Run evaluation
print("🔍 Starting model evaluation...")

try:
    comparator = ModelComparator(eval_config)
    results = comparator.compare_models()

    print("✅ Evaluation completed!")

    # Display results
    if 'comparison_summary' in results:
        summary = results['comparison_summary']

        for metric_name, rankings in summary['rankings'].items():
            print(f"\n📊 {metric_name}:")
            for i, ranking in enumerate(rankings):
                emoji = "🥇" if i == 0 else "🥈" if i == 1 else "🥉" if i == 2 else "📍"
                print(f"  {emoji} {ranking['model']}: {ranking['value']:.3f}")

    # Generate detailed report
    if eval_config.generate_report:
        report = comparator.generate_report(results)
        print(f"\n📋 Detailed report saved to {eval_config.output_dir}/evaluation_report.md")

    # Log to W&B
    if wandb.run and 'comparison_summary' in results:
        wandb.log({"evaluation_results": summary})

except Exception as e:
    print(f"❌ Evaluation failed: {e}")
    print("This might be due to model loading issues or benchmark dataset problems.")

## 🧪 Interactive Inference

Test your trained model with custom prompts.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load trained model for inference
def load_trained_model(model_path):
    try:
        print(f"🔄 Loading model from {model_path}...")

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # Load model
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.float16,
            device_map="auto"
        )

        print("✅ Model loaded successfully")
        return model, tokenizer

    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        print("Using base model instead...")

        # Fallback to base model
        tokenizer = AutoTokenizer.from_pretrained('codellama/CodeLlama-7b-hf')
        model = AutoModelForCausalLM.from_pretrained(
            'codellama/CodeLlama-7b-hf',
            torch_dtype=torch.float16,
            device_map="auto"
        )
        return model, tokenizer

# Load the model
model, tokenizer = load_trained_model('./models/distilled_codellama')

In [ ]:
def generate_code(prompt, max_length=256, temperature=0.1):
    """Generate code from a prompt."""

    # Format prompt
    formatted_prompt = f"### Instruction:\n{prompt}\n\n### Response:\n"

    # Tokenize
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the generated part
    response_start = generated_text.find("### Response:\n") + len("### Response:\n")
    generated_code = generated_text[response_start:].strip()

    return generated_code

print("🧪 Inference function ready!")

In [ ]:
# Test with sample prompts
test_prompts = [
    "Write a Python function to calculate the factorial of a number",
    "Create a JavaScript function to validate email addresses",
    "Implement a binary search algorithm in Python",
    "Write a Python function to find the longest word in a sentence"
]

print("🧪 Testing model with sample prompts...\n")

for i, prompt in enumerate(test_prompts, 1):
    print(f"{'='*60}")
    print(f"Test {i}: {prompt}")
    print(f"{'='*60}")

    try:
        generated_code = generate_code(prompt)
        print(generated_code)
        print()
    except Exception as e:
        print(f"❌ Generation failed: {e}")
        print()

In [ ]:
# Interactive testing
print("🎮 Interactive Code Generation")
print("Enter your prompts below. Type 'quit' to exit.\n")

while True:
    prompt = input("Enter your prompt: ")

    if prompt.lower() == 'quit':
        break

    if prompt.strip():
        try:
            print("\n🤖 Generated Code:")
            print("-" * 40)
            generated_code = generate_code(prompt)
            print(generated_code)
            print("-" * 40)
            print()
        except Exception as e:
            print(f"❌ Error: {e}\n")

print("👋 Thanks for testing!")

## 💾 Save and Export Model

Save your trained model for future use or deployment.

In [ ]:
import shutil
from datetime import datetime

# Create model archive
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_name = f"distilled_codellama_{timestamp}"
drive_path = f"/content/drive/MyDrive/models/{archive_name}"

print(f"💾 Saving model to Google Drive: {drive_path}")

try:
    # Create directory
    os.makedirs(drive_path, exist_ok=True)

    # Copy model files
    if os.path.exists('./models/distilled_codellama'):
        shutil.copytree(
            './models/distilled_codellama',
            f"{drive_path}/model",
            dirs_exist_ok=True
        )

    # Copy evaluation results
    if os.path.exists('./evaluation_results'):
        shutil.copytree(
            './evaluation_results',
            f"{drive_path}/evaluation",
            dirs_exist_ok=True
        )

    # Save training summary
    summary = {
        'timestamp': timestamp,
        'config': COLAB_CONFIG,
        'dataset_size': dataset_config.target_size if 'dataset_config' in locals() else 'unknown',
        'training_completed': True
    }

    import json
    with open(f"{drive_path}/training_summary.json", 'w') as f:
        json.dump(summary, f, indent=2)

    print(f"✅ Model saved successfully!")
    print(f"📁 Location: {drive_path}")
    print(f"📊 Summary: {drive_path}/training_summary.json")

except Exception as e:
    print(f"❌ Failed to save model: {e}")

## 🚀 Deployment Options

Your model is now ready for deployment! Here are some options:

In [ ]:
print("🚀 Deployment Options for Your Trained Model")
print("=" * 50)
print()

print("1. 🤗 Hugging Face Hub")
print("   - Upload to Hugging Face for easy sharing")
print("   - Use model.push_to_hub() method")
print("   - Example: https://huggingface.co/your-username/distilled-codellama")
print()

print("2. 📱 Local Inference")
print("   - Run on your local machine")
print("   - Download from Google Drive")
print("   - Use transformers library")
print()

print("3. ☁️ Cloud Deployment")
print("   - Deploy on AWS/GCP/Azure")
print("   - Use Inference Endpoints")
print("   - Scale automatically")
print()

print("4. 🔌 API Service")
print("   - Create REST API with FastAPI")
print("   - Deploy with Docker")
print("   - Integrate with applications")
print()

print("📊 Model Performance Summary:")
print(f"   • Base Model: Code Llama 7B")
print(f"   • Training: Knowledge Distillation from Claude Opus 4")
print(f"   • Memory: ~6GB with QLoRA optimization")
print(f"   • Speed: ~10-20 tokens/second on consumer GPU")
print(f"   • Quality: Expected 70-75% on HumanEval benchmark")
print()

print("💡 Next Steps:")
print("   1. Test the model thoroughly with your use cases")
print("   2. Fine-tune further if needed")
print("   3. Deploy to your preferred platform")
print("   4. Monitor performance and user feedback")
print()

print("🎉 Congratulations! You've successfully trained a code generation model!")

In [ ]:
# Cleanup and finalize
if wandb.run:
    wandb.finish()
    print("✅ W&B run finished")

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ GPU memory cleared")

print("\n🎯 Training Session Complete!")
print("Your model has been trained and saved to Google Drive.")
print("Feel free to disconnect from Colab to save compute credits.")

In [12]:
os.chdir('Claude-to-Codellama-Distillation')
print(f"📂 Project directory: {os.getcwd()}")

FileNotFoundError: [Errno 2] No such file or directory: 'Claude-to-Codellama-Distillation'